In [ ]:
%pip install google-cloud-language>=2.9.1
%pip install google-cloud-spanner








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.6/384.6 kB 5.9 MB/s eta 0:00:00


## Connect to Spanner

In [ ]:
# @title {display-mode: "form"}
PROJECT_ID = "span-cloud-testing"  # @param {type:"string"}

assert PROJECT_ID, "❌ Please enter your project ID"

print(f'✔️ PROJECT_ID: "{PROJECT_ID}"')

✔️ PROJECT_ID: "span-cloud-testing"


In [ ]:
from google.colab import auth

auth.authenticate_user(project_id=PROJECT_ID)

print(f"✔️ Authenticated")

✔️ Authenticated


In [ ]:
from google.cloud import spanner
import pandas as pd



["UPDATE EU_MutualFunds SET EU_MutualFunds.investment_strategy_Embedding = (SELECT embeddings.values   FROM ML.PREDICT(MODEL EmbeddingsModel,(SELECT EU_MutualFunds.investment_strategy AS content)) @{remote_udf_max_rows_per_rpc=200}) WHERE investment_strategy_Embedding is NULL and inception_date ='1933-03-24';\n", "UPDATE EU_MutualFunds SET EU_MutualFunds.investment_strategy_Embedding = (SELECT embeddings.values   FROM ML.PREDICT(MODEL EmbeddingsModel,(SELECT EU_MutualFunds.investment_strategy AS content)) @{remote_udf_max_rows_per_rpc=200}) WHERE investment_strategy_Embedding is NULL and inception_date ='1934-03-22';\n", "UPDATE EU_MutualFunds SET EU_MutualFunds.investment_strategy_Embedding = (SELECT embeddings.values   FROM ML.PREDICT(MODEL EmbeddingsModel,(SELECT EU_MutualFunds.investment_strategy AS content)) @{remote_udf_max_rows_per_rpc=200}) WHERE investment_strategy_Embedding is NULL and inception_date ='1937-01-01';\n", "UPDATE EU_MutualFunds SET EU_MutualFunds.investment_stra

In [ ]:
from google.cloud import spanner
from google.api_core.client_options import ClientOptions

# Your Cloud Spanner instance ID.
instance_id = "bagchi-mf-demo"

# Your Cloud Spanner database ID.
database_id = "mf-data"
options = ClientOptions(api_endpoint="staging-wrenchworks.sandbox.googleapis.com:443")

# Instantiate a client.
spanner_client = spanner.Client(client_options=options)


# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Get a Cloud Spanner database by ID.
database = instance.database(database_id)

def spanner_read_data(query):
  # Execute a simple SQL statement.
  outputs = []
  with database.snapshot() as snapshot:
    results = snapshot.execute_sql(query)
    rows = list()
    for row in results:
        rows.append(row)
  return rows

def spanner_read_data_withparam(query,vectorInput):
  # Execute a simple SQL statement.
  outputs = []
  with database.snapshot() as snapshot:

    results = snapshot.execute_sql(query,params={"vector": vectorInput},)
    rows = list()
    for row in results:
        rows.append(row)
    # Get column names
    cols = [x.name for x in results.fields]
    # Convert to pandas dataframe
    result_df = pd.DataFrame(rows, columns = cols)

  return result_df

In [ ]:
query = "SELECT embeddings. VALUES as vector FROM ML.PREDICT( MODEL EmbeddingsModel, (SELECT \"select companies which meet responsible investment criteria requirements on Environment, Social aspects.\" AS content) ) ;"
vectorInput = spanner_read_data(query)

len(vectorInput)
vectorInput[0][0]

# innerQuery = "SELECT NewMFSequence,fund_name, investment_strategy, investment_managers, APPROX_EUCLIDEAN_DISTANCE(investment_strategy_Embedding_vector, @vector, options => JSON '{\"num_leaves_to_search\": 10}') AS distance FROM EU_MutualFunds @{force_index = InvestmentStrategyEmbeddingIndex} WHERE investment_strategy_Embedding_vector IS NOT NULL ORDER BY distance LIMIT 100"
# returnInnerQuery = spanner_read_data_withparam(innerQuery,vectorInput[0][0])
# returnInnerQuery

intersectQuery="SELECT funds.fund_name, funds.investment_strategy, funds.investment_managers FROM (SELECT NewMFSequence, APPROX_EUCLIDEAN_DISTANCE(investment_strategy_Embedding_vector, @vector, options => JSON '{\"num_leaves_to_search\": 10}') AS distance FROM EU_MutualFunds @{force_index = InvestmentStrategyEmbeddingIndex} WHERE investment_strategy_Embedding_vector IS NOT NULL ORDER BY distance LIMIT 500 ) AS ann JOIN EU_MutualFunds AS funds ON ann.NewMFSequence = funds.NewMFSequence WHERE SEARCH_NGRAMS(funds.investment_managers_Substring_Tokens_NGRAM, 'Maarten',min_ngrams=>1) order by score_ngrams(funds.investment_managers_Substring_Tokens_NGRAM, 'Maarten') desc;"
intersectResults = spanner_read_data_withparam(intersectQuery,vectorInput[0][0])
intersectResults

# unionQuery="SELECT NewMFSequence FROM (SELECT NewMFSequence FROM EU_MutualFunds @{​force_index = InvestmentStrategyEmbeddingIndex} WHERE investment_strategy_Embedding_vector IS NOT NULL ORDER BY APPROX_EUCLIDEAN_DISTANCE(investment_strategy_Embedding_vector, @vector, options => JSON '{​\"num_leaves_to_search\": 10}') AS distance LIMIT 10 ) UNION DISTINCT (SELECT NewMFSequence FROM EU_MutualFunds WHERE SEARCH_NGRAMS(funds.investment_managers_Substring_Tokens_NGRAM, 'Louisa');"
# unionResults = spanner_read_data_withparam(unionQuery,vectorInput[0][0])
# unionResults


,fund_name,investment_strategy,investment_managers
0,Franklin Templeton Investment Funds - Templeto...,The Fund seeks to achieve its objective throug...,"Maarten Bloemen, Katherine Owen, Tina Sadler"
1,Franklin Templeton Investment Funds - Templeto...,The Fund seeks to achieve its objective throug...,"Maarten Bloemen, Katherine Owen, Tina Sadler"
2,Franklin Templeton Investment Funds - Templeto...,The Fund seeks to achieve its objective throug...,"Maarten Bloemen, Katherine Owen, Tina Sadler"
3,DWS Invest ESG Equity Income TFD,The objective of the investment policy of the ...,"Martin Berberich, Bertrand Born"
4,DWS Invest ESG Equity Income TFC,The objective of the investment policy of the ...,"Martin Berberich, Bertrand Born"
5,Federated Hermes Impact Opportunities Equity F...,The investment objective of the Fund is to ach...,"Ingrid Kukuljan, Martin Todd"
6,Federated Hermes Impact Opportunities Equity F...,The investment objective of the Fund is to ach...,"Ingrid Kukuljan, Martin Todd"
7,Federated Hermes Impact Opportunities Equity F...,The investment objective of the Fund is to ach...,"Ingrid Kukuljan, Martin Todd"
8,Federated Hermes Impact Opportunities Equity F...,The investment objective of the Fund is to ach...,"Ingrid Kukuljan, Martin Todd"
9,Federated Hermes Impact Opportunities Equity F...,The investment objective of the Fund is to ach...,"Ingrid Kukuljan, Martin Todd"
